In [1]:
# interactive widget imports
import ipywidgets as wg
from IPython.display import display


# data handling modules
import pandas as pd
import numpy as np
from datetime import datetime

# plotting imports
import seaborn as sb
import matplotlib.pyplot as plt
import plotly.plotly as py
import plotly.graph_objs as go
import plotly
from plotly import tools
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

plotly.tools.set_credentials_file(username='Flavioh', api_key='GogTSHQAuhgi5p724TsF')
plt.style.use('seaborn')

# web scraping
from bs4 import BeautifulSoup
import requests

# other helpers and suppress warnings
from helpers import *
from timeline_helpers import *
import warnings
warnings.filterwarnings("ignore", 'This pattern has match groups')

In [65]:
# Load data

df = pd.read_csv('../generated/FINAL_DF_WITH_TOPICS.csv', index_col=0)
df['publish_date'] = pd.to_datetime(df['publish_date'])
df = df[(df.publish_date > '2015-09-01') & (df.publish_date < '2016-11-10')]

/anaconda3/envs/ada/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning:

Columns (6,12,23,24) have mixed types. Specify dtype option on import or set low_memory=False.



In [66]:
df.head()

,tweet_text,author,publish_date,following,followers,post_type,account_category,userid,tweetid,tweet_time,...,retweet_count,hashtags,urls,user_mentions,poll_choices,follower_count,following_count,last_tweet_at,topic,topic_scores
0,#FoodPoisoning is not a joke! #Walmart #KochFa...,1D_NICOLE_,2015-11-26 22:20:00,48,40,RETWEET,Fearmonger,d077a7eaf7286285c8c406b3d5a6449b51e5ee81d26b5a...,670002488175628293,2015-11-26 22:13:00,...,15,"[FoodPoisoning, Walmart, KochFarms]",[http://www.discusscooking.com/forums/f26/very...,NaN,NaN,107,124,2015-12-15 18:06,Trump Support,[-8.1184225 -4.818271 -2.6408563 -0.574982...
1,#FoodPoisoning is not a joke! #Walmart #KochFa...,BARELYBARBOZA,2015-11-26 22:34:00,50,48,RETWEET,Fearmonger,d077a7eaf7286285c8c406b3d5a6449b51e5ee81d26b5a...,670002488175628293,2015-11-26 22:13:00,...,15,"[FoodPoisoning, Walmart, KochFarms]",[http://www.discusscooking.com/forums/f26/very...,NaN,NaN,107,124,2015-12-15 18:06,Trump Support,[-8.1184225 -4.818271 -2.6408563 -0.574982...
2,#FoodPoisoning is not a joke! #Walmart #KochFa...,D_ANDRE_AUSTIN_,2015-11-26 22:20:00,0,49,RETWEET,Fearmonger,d077a7eaf7286285c8c406b3d5a6449b51e5ee81d26b5a...,670002488175628293,2015-11-26 22:13:00,...,15,"[FoodPoisoning, Walmart, KochFarms]",[http://www.discusscooking.com/forums/f26/very...,NaN,NaN,107,124,2015-12-15 18:06,Trump Support,[-8.1184225 -4.818271 -2.6408563 -0.574982...
3,#FoodPoisoning is not a joke! #Walmart #KochFa...,ERTMANGRETA,2015-11-26 22:20:00,57,50,RETWEET,RightTroll,d077a7eaf7286285c8c406b3d5a6449b51e5ee81d26b5a...,670002488175628293,2015-11-26 22:13:00,...,15,"[FoodPoisoning, Walmart, KochFarms]",[http://www.discusscooking.com/forums/f26/very...,NaN,NaN,107,124,2015-12-15 18:06,Trump Support,[-8.1184225 -4.818271 -2.6408563 -0.574982...
4,#FoodPoisoning is not a joke! #Walmart #KochFa...,FREDYTHECREATOR,2015-11-26 22:34:00,65,50,RETWEET,Fearmonger,d077a7eaf7286285c8c406b3d5a6449b51e5ee81d26b5a...,670002488175628293,2015-11-26 22:13:00,...,15,"[FoodPoisoning, Walmart, KochFarms]",[http://www.discusscooking.com/forums/f26/very...,NaN,NaN,107,124,2015-12-15 18:06,Trump Support,[-8.1184225 -4.818271 -2.6408563 -0.574982...


In [67]:
data_filtered = df[['publish_date', 'account_category', 'tweet_text', 'followers', 'following', 'author', 'like_count', 'topic']]

In [68]:
data_filtered.topic.value_counts()

Black Support        147872
Trump Support        115779
Sports               113940
Entertainment         83278
Health                42000
Foreign Countries     24942
Crime                 20898
Anti-Islam            18982
Anti-Trump            16452
Patriot               11349
Fukushima               554
Name: topic, dtype: int64

In [70]:
right_trolls = data_filtered[data_filtered.account_category == 'RightTroll']
left_trolls = data_filtered[data_filtered.account_category == 'LeftTroll']
fearmonger = data_filtered[data_filtered.account_category == 'Fearmonger']

In [71]:
import matplotlib.pyplot as plt

import numpy as np
%matplotlib inline

def count_topic(topic, df):
    """The function takes in a language as a DEFINE_string
    and goes through a dataframe that has one column named
    topic and another one that has publish_date. It then
    groups the tweets by month and returns the sum of the
    tweets during the time period"""

    filt = df[df.topic == topic].copy()
    filt['topic_num'] = filt.topic.map({topic:1})
    return filt.groupby(pd.Grouper(key='publish_date', freq='1D')).sum()

def trace_generator_topic(topic_df):
    """This function generates the data that will be used
    as input for the iplot function. It prepares the labels
    from the index"""
    data = []
    for topic in topic_df.topic.value_counts().index.values:
        filtered=count_topic(topic, topic_df)
        strd = pd.Series(filtered.index.strftime('%Y-%m-%d %H-%M-%S'))
        xlabels = list(strd.apply(lambda x: x[0:10]))
        trace = go.Scatter(x=xlabels,
                            y=filtered.topic_num.values,
                            fill='tozeroy',
                            mode= 'none',
                            name=topic)
        data.append(trace)

    return data


In [72]:
data = trace_generator_topic(right_trolls)
layout = do_layout('Date', 'Number of Tweets', 'Topic as a Function of Time')
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='x')

In [73]:
data = trace_generator_topic(left_trolls)
layout = do_layout('Date', 'Number of Tweets', 'Topic as a Function of Time')
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='y')

In [55]:
data_filtered[data_filtered.topic == 'Anti-Trump']

,publish_date,tweet_text,followers,following,author,like_count,topic,value
3146,2015-12-16 01:10:00,#Cruz is the worst candidate for our country #...,60,76,4EVER_SUSAN,0,Anti-Trump,1.0
3147,2015-12-16 01:33:00,#Cruz is the worst candidate for our country #...,62,59,AMALIA_PETTY,0,Anti-Trump,1.0
3148,2015-12-16 01:48:00,#Cruz is the worst candidate for our country #...,68,100,_SOLOMONALBERT_,0,Anti-Trump,1.0
3268,2016-01-24 23:26:00,"In the coming months, we'll know if those to b...",2183,4744,4MYSQUAD,11,Anti-Trump,1.0
3317,2015-10-01 06:44:00,#Sheriff that Laughed about Flash Banging a To...,879,1440,4MYSQUAD,0,Anti-Trump,1.0
3334,2015-10-11 22:10:00,Photo taken by the at the #MillionMamMarch #B...,1105,2001,4MYSQUAD,6,Anti-Trump,1.0
3373,2015-10-12 14:46:00,last year perfect twitt https://t.co/1q23ihihUV,1128,1774,4MYSQUAD,0,Anti-Trump,1.0
3446,2015-10-13 09:01:00,True #BlackPeopleProblems #Blackpeoplebelike #...,1147,2001,4MYSQUAD,4,Anti-Trump,1.0
3487,2015-10-15 21:12:00,"#NYPD Destroyed birth certificates, medication...",1207,1979,4MYSQUAD,3,Anti-Trump,1.0
3587,2015-10-18 22:50:00,Im not feeling it today. Kinda sick of it toda...,1248,1618,4MYSQUAD,1,Anti-Trump,1.0


# Examination of Top Authors

In [115]:
data_filtered[data_filtered.author == 'BRIANAREGLAND']

,publish_date,account_category,tweet_text,followers,following,author,like_count,topic,value
5989,2016-02-09 09:04:00,LeftTroll,Youth program set to open March 2016! Follow us,205,400,BRIANAREGLAND,0,Sports,1.0
5990,2016-02-09 09:04:00,LeftTroll,Youth program set to open March 2016! Follow us,205,400,BRIANAREGLAND,0,Sports,1.0
5991,2016-02-09 09:04:00,LeftTroll,Youth program set to open March 2016! Follow us,205,400,BRIANAREGLAND,0,Sports,1.0
5992,2016-02-09 09:04:00,LeftTroll,Youth program set to open March 2016! Follow us,205,400,BRIANAREGLAND,0,Sports,1.0
5993,2016-02-09 09:04:00,LeftTroll,Youth program set to open March 2016! Follow us,205,400,BRIANAREGLAND,0,Sports,1.0
5994,2016-02-09 09:04:00,LeftTroll,Youth program set to open March 2016! Follow us,205,400,BRIANAREGLAND,0,Sports,1.0
5995,2016-02-09 09:04:00,LeftTroll,Youth program set to open March 2016! Follow us,205,400,BRIANAREGLAND,0,Sports,1.0
5996,2016-02-09 09:04:00,LeftTroll,Youth program set to open March 2016! Follow us,205,400,BRIANAREGLAND,0,Sports,1.0
5997,2016-02-09 09:04:00,LeftTroll,Youth program set to open March 2016! Follow us,205,400,BRIANAREGLAND,0,Sports,1.0
5998,2016-02-09 09:04:00,LeftTroll,Youth program set to open March 2016! Follow us,205,400,BRIANAREGLAND,0,Sports,1.0


In [117]:
# Extract top 10 posting authors
top_authors = data_filtered.author.value_counts()[0:40]

# Filter data frame for them
###### testing with screamymonkey

topauthors = data_filtered[data_filtered.author.isin(top_authors.index.values)]
test = topauthors.copy()
# test = data_filtered[data_filtered.author == 'BRIANAREGLAND']


In [ ]:
test.head()

In [110]:
top_authors

SCREAMYMONKEY      39082
MILWAUKEEVOICE     35381
PHOENIXDAILYNEW    33211
ROOMOFRUMOR        26774
TODAYPITTSBURGH    19880
SEATTLE_POST       18816
KANSASDAILYNEWS    17490
DAILYSANJOSE       14070
DAILYSANFRAN       13812
NEWSPEAKDAILY       9483
BATONROUGEVOICE     9464
DAILYSANDIEGO       9168
NEWORLEANSON        8889
NEWARKVOICE         8795
OAKLANDONLINE       8295
SANANTOTOPNEWS      7155
JENN_ABRAMS         6812
STLOUISONLINE       6434
WASHINGTONLINE      6425
TODAYINSYRIA        5948
WORLDOFHASHTAGS     5522
CHICAGODAILYNEW     5009
JACKSONCITYPOST     4753
RICHMONDVOICE       4436
DANAGEEZUS          4347
TRAYNESHACOLE       4312
PIGEONTODAY         3895
ONLINECLEVELAND     3770
BLEEPTHEPOLICE      3536
BOOTH_PRINCE        3478
TODAYBOSTONMA       3281
TEN_GOP             3250
ANDYHASHTAGGER      3168
CHRIXMORGAN         2843
THEFOUNDINGSON      2677
TODAYCLEVELAND      2622
DARKNALLY           2346
CORYNAENAE          2345
CLAYPAIGEBOO        2189
BRIANAREGLAND       2121


In [118]:

data1 = trace_generator_topic(test)
layout = do_layout('Date', 'Number of Tweets', 'Topic for Top 10 Authors Over Time')
fig = go.Figure(data=data1, layout=layout)
py.iplot(fig, filename='y')

In [16]:
# Check out follower counts for top 10 hashtaggers

group_followers = test.groupby([pd.Grouper(key='publish_date', freq='1D'), 'author'])['followers'].max().groupby(level=0).sum()

######

test['tweet_counts'] = np.ones(test.shape[0])

group_likes =test.groupby([pd.Grouper(key='publish_date', freq='1D'),
                                 'author'])['like_count'].max().groupby(level=0).sum()

group_following =test.groupby([pd.Grouper(key='publish_date', freq='1D'),
                                 'author'])['following'].max().groupby(level=0).sum()
group_authors_time = pd.DataFrame({'followers': group_followers.values,
                                   'following': group_following.values},
                                  index=group_followers.index.values)

 
# group by month
general_timeline = test.groupby(pd.Grouper(key='publish_date', freq='1D')).sum()
# construct labels
xlabels = list(pd.Series(group_authors_time.index.strftime(
    '%Y-%m-%d %H-%M-%S')).apply(lambda x: x[0:10]))
data = []

for col in group_authors_time:
    filtered = group_authors_time[col].copy()
    trace = go.Scatter(x=xlabels, y=filtered.values, name=col, 
                       fill='tozeroy', mode='lines')
    data.append(trace)

filtered = general_timeline['tweet_counts'].copy()
trace=go.Scatter(x=xlabels, y=filtered.values, marker = {'color' : '#00AA00'}, name='Tweet Counts', fill='tozeroy', mode='lines')
data.append(trace)
trace = go.Scatter(x=xlabels, y=group_likes.values, marker = {'color' : '#FF2200'}, name='Tweet Likes', fill='tozeroy', mode='lines')
data.append(trace)



# Create plots for following, followers, updates, tweet_counts

fig = tools.make_subplots(rows=3, cols=1)

fig.append_trace(data[0], 1, 1)
fig.append_trace(data[1], 1, 1)
fig.append_trace(data[2], 2, 1)
fig.append_trace(data[3], 3, 1)

fig['layout'].update(height=600, width=800, title='General Trends Across Time')
py.iplot(fig, filename='y')


This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (3,1) x3,y3 ]



In [ ]:
# Calculate correlation values

# Creating dataframe of entries for correlation statistics
corr_df = pd.DataFrame({'Followers': group_followers.values, 'Likes': group_likes.values,
                       'Following': group_following.values})
corr_df.index = xlabels                        

topics = data_filtered.topic.value_counts().index.values
for topic in topics:
    x = count_topic(topic, topauthors)
    corr_df = corr_df.join(x[['topic_num']], how='outer', rsuffix=topic)

corr_df.columns = ['Followers', 'Likes', 'Following', *topics]

corr_df.fillna(0, inplace=True)


In [ ]:
print(corr_df.iloc[0:200].Sports.corr(corr_df.iloc[0:200].Followers))
print(corr_df.iloc[0:200]['w'].corr(corr_df.iloc[0:200].Followers))
print(corr_df.iloc[0:200]['Entertainment'].corr(corr_df.iloc[0:200].Followers))

In [ ]:
print(corr_df.iloc[0:200].corr().iloc[:,0])
print(corr_df.iloc[-100:].corr().iloc[:,0])

# Swing State Stuff

In [21]:
import requests
from bs4 import BeautifulSoup
import lxml

#https://en.wikipedia.org/wiki/List_of_largest_cities_of_U.S._states_and_territories_by_population
# Scrape wikipedia table for american cities and states
website = requests.get('https://simple.wikipedia.org/wiki/List_of_U.S._states%27_largest_cities').text
soup = BeautifulSoup(website,'html.parser')
My_table = soup.find('table',{'class':'wikitable sortable mw-collapsible'})
links = My_table.find_all('a')

places = []
for link in links:
    places.append(link.get('title'))
    
city_clean = list(filter(None.__ne__, places)) # Drop None values
cities = {}


for entry in city_clean:
    split=entry.split(', ')
    if len(split) == 1:
        cities[split[0]] = []
    else:
        if split[1] in cities.keys():
            cities[split[1]].append(split[0]) 
state_n_city = []
for state in cities.keys():
    state_list = []
    state_cities = cities[state]
    for city in state_cities:
        state_list.append(city + '|' + city.lower())
    state_list.append(state + '|' + state.lower())
    state_list = '|'.join(state_list)
    state_n_city.append([state, state_list])
    
state_n_city_dict = {}
for state in state_n_city:
    state_n_city_dict[state[0]] = state[1]

state_n_city_dict['Georgia (U.S. state)'] += '|Georgia|georgia'




In [33]:
abbreviations = ['AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'FL',
                                  'GA', 'HI','ID','IL','IN','IA','KS','KY','LA','ME', 'MD', 'MA',
                                  'MI','MN','MS','MO','MT','NE','NV','NH','NJ','NM','NY',
                                  'NC','ND','OH','OK','OR','PA','RI','SC','SD','TN','TX',
                                  'UT','VT','VA','WA','WV','WI', 'WY']

In [ ]:
# Load old data set

election_month = pd.read_csv('../generated/onemonth_dataset.csv', index_col=0)
election_month['publish_date'] = pd.to_datetime(election_month['publish_date'])
election_month = election_month[election_month.language == 'English']
election_month = election_month[['author', 'content', 'publish_date', 'followers', 'following', 'topic']]
data_filtered['value'] = np.ones(data_filtered.shape[0])

election_month['value'] = np.ones(election_month.shape[0])

In [101]:
data_filtered['value'] = np.ones(data_filtered.shape[0])

/anaconda3/envs/ada/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [23]:
swing_states = state_n_city_dict.keys()

In [102]:
# # for each swing state, calculate the proportions of the different topic categories
# for state in swing_states:
#     filtered = election_month[election_month.content.str.contains(state_n_city_dict[state])].copy()
#     print(state)
#     print(filtered.groupby('topic')['value'].sum()/filtered.groupby('topic')['value'].sum().sum() * 100)
black_support = []
trump_support = []
numbers = []
for state in swing_states:
    filtered = data_filtered[data_filtered.tweet_text.str.contains(state_n_city_dict[state])].copy()
#     print(state)
    numbers.append(filtered.shape[0])
    percents = filtered.groupby('topic')['value'].sum()/filtered.groupby('topic')['value'].sum().sum() * 100
#     print(percents.loc['Black Support'])
    black_support.append(percents.loc['Black Support'])
    trump_support.append(percents.loc['Trump Support'])
#     print('\n')

In [103]:
black = pd.DataFrame()
trump = pd.DataFrame()

In [104]:
black['percent_black_support'] = black_support
black['state'] = abbreviations
black['numbers'] = numbers

trump['percent_trump_support'] = trump_support
trump['state'] = abbreviations
trump['numbers'] = numbers

In [63]:

data = [ dict(
        type='choropleth',
        colorscale = [[1.0, 'rgb(49,54,149)'],
                      [0.8888888888888888, 'rgb(69,117,180)'],
                      [0.7777777777777778, 'rgb(116,173,209)'],
                      [0.6666666666666666, 'rgb(171,217,233)'], 
                      [0.5555555555555556, 'rgb(224,243,248)'],
                      [0.4444444444444444, 'rgb(254,224,144)'],
                      [0.3333333333333333, 'rgb(253,174,97)'],
                      [0.2222222222222222, 'rgb(244,109,67)'],
                      [0.1111111111111111, 'rgb(215,48,39)'],
                      [0.0, 'rgb(165,0,38)']],
        autocolorscale = False,
        locations = black['state'],
        z = black['percent_black_support'].astype(float).apply(lambda x: np.log10(x))
,
        locationmode = 'USA-states',
        text = black['numbers'],
        marker = dict(
            line = dict (
                color = 'rgb(255,255,255)',
                width = 2
            ) ),
        colorbar = dict(
            title = "Number of Tweets (log-value)")
        ) ]

layout = dict(
        title = 'Black Percentage According to State',
        geo = dict(
            scope='usa',
            projection=dict( type='albers usa' ),
            showlakes = True,
            lakecolor = 'rgb(255, 255, 255)'),
             )
    
fig = dict( data=data, layout=layout )
py.iplot( fig, filename='blackstuff' )

In [116]:

data = [ dict(
        type='choropleth',
        colorscale = [[1.0, 'rgb(49,54,149)'],
                      [0.8888888888888888, 'rgb(69,117,180)'],
                      [0.7777777777777778, 'rgb(116,173,209)'],
                      [0.6666666666666666, 'rgb(171,217,233)'], 
                      [0.5555555555555556, 'rgb(224,243,248)'],
                      [0.4444444444444444, 'rgb(254,224,144)'],
                      [0.3333333333333333, 'rgb(253,174,97)'],
                      [0.2222222222222222, 'rgb(244,109,67)'],
                      [0.1111111111111111, 'rgb(215,48,39)'],
                      [0.0, 'rgb(165,0,38)']],
        autocolorscale = False,
        locations = trump['state'],
        z = trump['percent_trump_support'].astype(float)
,
        locationmode = 'USA-states',
        text = trump['numbers'],
        marker = dict(
            line = dict (
                color = 'rgb(255,255,255)',
                width = 2
            ) ),
        colorbar = dict(
            title = "Number of Tweets (log-value)")
        ) ]

layout = dict(
        title = 'Trump Percentage According to State',
        geo = dict(
            scope='usa',
            projection=dict( type='albers usa' ),
            showlakes = True,
            lakecolor = 'rgb(255, 255, 255)'),
             )
    
fig = dict( data=data, layout=layout )
py.iplot( fig, filename='trumpstuff' )